In [1]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import implicit
import numpy as np
import scipy
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)

In [3]:
data = df.drop(['Unnamed: 0','artist_name'], axis=1)

In [4]:
data.user_id = data.user_id.factorize()[0]

In [5]:
users = list(np.sort(data.user_id.unique()))
artists = list(data.artist_mbid.unique())
plays = list(data.plays)

In [6]:
#user-user
rows2 = data.user_id.astype('category', categories=users).cat.codes
cols2 = data.artist_mbid.astype('category', categories=artists).cat.codes
plays_sparse2 = scipy.sparse.csr_matrix((plays, (rows2, cols2)), shape=(len(users),len(artists)))

In [7]:
#item-item
rows = data.artist_mbid.astype('category', categories=artists).cat.codes
cols = data.user_id.astype('category', categories=users).cat.codes
plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(artists),len(users)))

In [50]:
sample_artists = ['kanye west',
'coldplay',
'justin timberlake']

"""
'death cab for cutie',
'paramore',
'ok go',
'foo fighters',
'green day',
'eminem',
'kanye west',
'maroon 5',
'britney spears',
'justin timberlake',
'notorious b.i.g.'
"""

"\n'death cab for cutie',\n'paramore',\n'ok go',\n'foo fighters',\n'green day',\n'eminem',\n'kanye west',\n'maroon 5',\n'britney spears',\n'justin timberlake',\n'notorious b.i.g.'\n"

In [ ]:
"""
for artist in sample_artists:
    artist_index = df.artist_name[df.artist_name == artist].index.tolist()[0]
    model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
    model_knn.fit(plays_sparse)
    distances, indices = model_knn.kneighbors(plays_sparse[artist_index], n_neighbors = 6)
    print('Artists that are similar to', artist + ':')
    for i in range(0, len(indices[0])):
        print(df.artist_name[df.artist_mbid == artists[indices[0][i]]].unique())
    print('\n')
"""

In [64]:
model_knn = implicit.nearest_neighbours.BM25Recommender()
model_knn.fit(plays_sparse, show_progress=True)

100%|██████████| 47102/47102 [00:00<00:00, 129377.60it/s]


In [51]:
for artist in sample_artists:
    artist_index = df.artist_name[df.artist_name == artist].index.tolist()[0]
    related = model_knn.similar_items(artist_index)
    print('Artists that are similar to', artist + ':')
    for (x,y) in related:
        print(df.artist_name[df.artist_mbid == artists[x]].unique())
    print('\n')

Artists that are similar to kanye west:
['charles mingus']
['miles davis']
['john coltrane']
['thelonious monk']
['duke ellington']
['charlie parker']
['herbie hancock']
['sonny rollins']
['bill evans']
['tom waits']


Artists that are similar to coldplay:
['coldplay']
['radiohead']
['the killers']
['red hot chili peppers']
['muse']
['u2']
['the beatles' 'beatles']
['plan x' 'kiraç & funda arar - www.catlaemi.net'
 'bisbal y alejandro sanz' ... 'frank sinatra/duke ellington'
 'chase kreger' 'martingo']
['snow patrol']
['death cab for cutie']


Artists that are similar to justin timberlake:
['inxs']
['depeche mode']
['u2']
['the cure']
['coldplay']
['queen']
['pink floyd']
['alanis morissette']
['david bowie']
['simple minds']




In [58]:
plays_sparse

<47102x9000 sparse matrix of type '<class 'numpy.int64'>'
	with 438337 stored elements in Compressed Sparse Row format>

In [55]:
model = implicit.als.AlternatingLeastSquares(factors=50)
model.fit(plays_sparse)

100%|██████████| 15.0/15 [00:03<00:00,  4.04it/s]


In [56]:
user_plays_sparse = plays_sparse.T.tocsr()
user_plays_sparse

<9000x47102 sparse matrix of type '<class 'numpy.int64'>'
	with 438337 stored elements in Compressed Sparse Row format>

In [68]:
user_plays_sparse = plays_sparse.T.tocsr()
model.recommend(1000,user_plays_sparse, N=20)

[(1569, 1.8930104),
 (1076, 1.693833),
 (4648, 1.6386454),
 (1074, 1.5217786),
 (5695, 1.4756634),
 (1339, 1.4285048),
 (351, 1.426858),
 (7086, 1.4050913),
 (363, 1.3643575),
 (120, 1.3570861),
 (5741, 1.3404362),
 (275, 1.3284478),
 (3632, 1.3030187),
 (336, 1.2511883),
 (1741, 1.246616),
 (1226, 1.2420986),
 (679, 1.2226174),
 (781, 1.2172772),
 (5010, 1.2172517),
 (85, 1.2031524)]

In [52]:
user_plays_sparse

<9000x47102 sparse matrix of type '<class 'numpy.int64'>'
	with 438337 stored elements in Compressed Sparse Row format>

In [18]:
model_knn.rank_items(int(users[1]),plays_sparse2,artists[1:3])

TypeError: '>=' not supported between instances of 'str' and 'int'